In [ ]:
import sys
sys.path.append('../src/wadiroscnn')
import hyperopt as hp
import numpy as np
import scipy as sc
import models as hm
import benchmark_functions as bf
import benchmarking_functions as bench
from sklearn.preprocessing import StandardScaler
import sklearn as sk
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
import mlflow
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from datetime import datetime
import torch.nn as nn
import cvxpy as cp
import mlflow.pytorch
from mlflow.models.signature import infer_signature, set_signature
import hyperopt
from functools import partial
import pandas as pd
import seaborn as sns

First, run this from terminal :

mlflow server --host 127.0.0.1 --port 8081

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8081")
# Create a new MLflow Experiment
experiment = mlflow.set_experiment("name_experiment")

# Get Experiment Details
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Name: {}".format(experiment.name))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

In [ ]:


search_space_DR_SCNN = {'radius': hp.loguniform('radius',-8, 1),
                'max_neurons': hp.quniform('max_neurons', 10, 300, 1),
                'wasserstein': hp.choice('wasserstein', ["l1", "l2"]),
                }

search_space_SCNN = {'lamb_reg': hp.loguniform('lamb_reg',-8, 1),
                'max_neurons': hp.quniform('max_neurons', 10, 300, 1),
                'regularizer': hp.choice('regularizer', ["LASSO", "RIDGE"]),
                }

search_space_SCNN_no_reg = {#'lamb_reg': hp.loguniform('lamb_reg',-8, 1),
                'max_neurons': hp.quniform('max_neurons', 10, 300, 1),
                }


search_space_DR_linreg = {'radius': hp.loguniform('radius',-8, 1),
                          'wasserstein': hp.choice('wasserstein', ["l1", "l2"]),
                }

search_space_linreg = {'lamb_reg': hp.loguniform('lamb_reg',-8, 1),
                'regularizer': hp.choice('regularizer', ["LASSO","RIDGE"]),
                }

search_space_FNN =  {'batch_size' : hp.quniform('batch_size',2, 100, 1),
                'learning_rate': hp.loguniform('learning_rate',-8, 1),
                'n_hidden' : hp.quniform('n_hidden',10, 300, 1),
                'n_epochs': hp.quniform( 'n_epochs',1, 1000, 1),
                'dropout_p': hp.uniform('dropout_p', 0.01, 0.4)}


In [ ]:
rng = np.random.default_rng(42)
criterion = nn.L1Loss()

solver_name = "CLARABEL"
max_evals = 400
max_evals_FNN = int(max_evals/4)
verbose = False
N = 2000
train_size = 0.75
test_size = 0.2
n_corrupted_points = int(N*0.8*0.1) #int(0.6*N)
n_dim = 4

FUNCTIONS_LIST = [ 
	[bf.Ackley, n_dim, "Ackley"],			
	[bf.Keane,n_dim, "Keane"],
	[bf.PichenyGoldsteinAndPrice,2, "PichenyGoldsteinAndPrice"],
	[bf.McCormick, 2, "McCormick"], # 2 dims
	]

In [ ]:
for benchmark_func in FUNCTIONS_LIST:
    data = bench.dataset_wrapper(benchmark_func=benchmark_func[0], n_dim=benchmark_func[1], rng =rng, N = N, train_size=train_size, test_size=test_size)
    func_name = benchmark_func[2]
    data.generate_data_wasserstein_corrupt_both(corrupt_data_points= n_corrupted_points, min_distance=0.05, n_projections=100, k_multiplier=1.5, L=2, seed=42)


    print(f"SCNN_no_reg on {func_name}: \n")
    fmin_SCNN_no_reg = partial(bench.objective_SCNN_no_reg, data = data, criterion = criterion, solver_name = solver_name, experiment=experiment, n_corrupted_points = n_corrupted_points, func_name=func_name, verbose=verbose)
    argmin_SCNN_no_reg = fmin(fn=fmin_SCNN_no_reg,
              space=search_space_SCNN_no_reg,
              algo= hyperopt.anneal.suggest, #hyperopt.tpe.suggest, # try anneal.suggest
              max_evals=int(max_evals)) #trials=spark_trials)
    
    print(f"WaDiRO SCNN on {func_name}: \n")
    fmin_DR_SCNN = partial(bench.objective_DR_SCNN, data = data, criterion = criterion, solver_name = solver_name, experiment=experiment, n_corrupted_points=n_corrupted_points, func_name=func_name, verbose=verbose)
    argmin_DR_SCNN = fmin(fn=fmin_DR_SCNN,
              space=search_space_DR_SCNN,
              algo= hyperopt.anneal.suggest, #hyperopt.tpe.suggest, # try anneal.suggest
              max_evals=max_evals) #trials=spark_trials)
    
    print(f"SCNN on {func_name}: \n")
    fmin_SCNN = partial(bench.objective_SCNN, data = data, criterion = criterion, solver_name = solver_name, experiment=experiment, n_corrupted_points = n_corrupted_points, func_name=func_name, verbose=verbose)
    argmin_SCNN = fmin(fn=fmin_SCNN,
              space=search_space_SCNN,
              algo= hyperopt.anneal.suggest, #hyperopt.tpe.suggest, # try anneal.suggest
              max_evals=max_evals) #trials=spark_trials)
    
    print(f"linreg on {func_name}: \n")
    fmin_linreg = partial(bench.objective_linreg, data = data, criterion = criterion, solver_name = solver_name, experiment=experiment, n_corrupted_points=n_corrupted_points,  func_name=func_name, verbose=verbose)
    argmin_linreg = fmin(fn=fmin_linreg,
              space=search_space_linreg,
              algo= hyperopt.anneal.suggest,#hyperopt.tpe.suggest, # try anneal.suggest
              max_evals=max_evals) #trials=spark_trials)

    print(f"WaDiRo linreg on {func_name}: \n")
    fmin_DR_linreg = partial(bench.objective_DR_linreg, data = data, criterion = criterion, solver_name = solver_name, experiment=experiment, n_corrupted_points=n_corrupted_points,  func_name=func_name, verbose=verbose)
    argmin_DR_linreg = fmin(fn=fmin_DR_linreg,
              space=search_space_DR_linreg,
              algo= hyperopt.anneal.suggest,#hyperopt.tpe.suggest, # try anneal.suggest
              max_evals=max_evals) #trials=spark_trials)
              
    
    print(f"FNN on {func_name}: \n")
    fmin_FNN = partial(bench.objective_FNN, data = data, criterion = criterion, experiment=experiment, n_corrupted_points=n_corrupted_points,  func_name=func_name, verbose=verbose)
    argmin_FNN = fmin(fn=fmin_FNN,
              space=search_space_FNN,
              algo= hyperopt.anneal.suggest, # try anneal.suggest
              max_evals=max_evals_FNN) #trials=spark_trials)


# Extract information

In [ ]:
df = mlflow.search_runs([experiment.experiment_id])
#building_ids = df['params.building_id']
display(df)
functions = df['params.benchmark_functions'].unique()
print(functions)
models = ['DR_SCNN',  'SCNN', 'SCNN_no_reg', 'FNN',  'DR_linreg', 'linreg' ]

In [ ]:
import json
result_dict = {}

for function in functions:
    if function != 'DeJong5' and function != 'Schwefel' and function != None:
        for model in models:
            best_mae = df.loc[(df['params.benchmark_functions'] == function) & (df['tags.model_name'] == model)].sort_values(by='metrics.MAE_val', ascending=True).head(1)
            best_mse = df.loc[(df['params.benchmark_functions'] == function) & (df['tags.model_name'] == model)].sort_values(by='metrics.MSE_val', ascending=True).head(1)
            #best_build = df.loc[ (df['tags.model_name'] == model)].sort_values(by='metrics.MAE_VAL_all_buildings_scaled', ascending=True).head(1)

            if f'{function}' not in result_dict.keys():
                result_dict[f'{function}'] = {}
        
            #result_dict[f'{function}'][f'{model}'] = {'best_mae': best_mae['metrics.MAE_test'].values[0], 'best_mse': best_mse['metrics.MSE_test'].values[0]}
            result_dict[f'{function}'][f'{model}'] = {'best_mae': best_mae['metrics.MAE_test'].values[0], 'best_mse': best_mae['metrics.MSE_test'].values[0], 'run_id': best_mae['run_id'].values[0]}


        

In [ ]:
print(result_dict)

In [ ]:
pd.DataFrame.from_dict(result_dict)

In [ ]:
df_2 = pd.concat({
        k: pd.DataFrame.from_dict(v, 'index') for k, v in result_dict.items()
    }, 
    axis=0)

In [ ]:
df_2['best_mae']['Ackley'].values

In [ ]:
data = pd.DataFrame({
    'Model': ['WaDiRo SCNN', 'SCNN with regularization',  'SCNN no regularization', 'Deep FNN', 'WaDiRo linear reg.', 'Linear reg. with regularization',  ],
    'McCormick MAE': df_2['best_mae']['McCormick'].values,
    'McCormick MSE': df_2['best_mse']['McCormick'].values,
    'PGandP MAE': df_2['best_mae']['PichenyGoldsteinAndPrice'].values,
    'PGandP MSE': df_2['best_mse']['PichenyGoldsteinAndPrice'].values,
    'Keane MAE': df_2['best_mae']['Keane'].values,
    'Keane MSE': df_2['best_mse']['Keane'].values,
    'Ackley MAE': df_2['best_mae']['Ackley'].values,
    'Ackley MSE':  df_2['best_mse']['Ackley'].values
}).set_index('Model')

# Normalize each column by its range
data_norm = (data - data.min()) / (data.max() - data.min())

plt.figure(figsize=(10, 6), dpi = 400)
#plt.tight_layout()
cmap = sns.color_palette("coolwarm", as_cmap=True)
heatmap = sns.heatmap(data_norm, cmap=cmap, annot=True, fmt=".2f",cbar_kws={'label': 'Normalized error'})

# Set the function names as the y-axis labels
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)

# Set the function names as the x-axis labels and place them on top
heatmap.set_xticklabels(data.columns, rotation=45)
heatmap.xaxis.tick_top()

#plt.title('Your caption here', y=-0.1)
#plt.title('Normalized error metrics for different models and benchmark functions')
plt.savefig('heatmap_JOPT.pdf', bbox_inches='tight')
plt.show()

In [ ]:


plt.figure(figsize=(10, 6), dpi = 400)
#plt.tight_layout()
cmap = sns.color_palette("coolwarm", as_cmap=True)
heatmap = sns.heatmap(data, cmap=cmap, annot=True, fmt=".2f",cbar_kws={'label': 'Error'})

# Set the function names as the y-axis labels
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)

# Set the function names as the x-axis labels and place them on top
heatmap.set_xticklabels(data.columns, rotation=45)
heatmap.xaxis.tick_top()

#plt.title('Your caption here', y=-0.1)
#plt.title('Normalized error metrics for different models and benchmark functions')
plt.savefig('heatmap_JOPT_corruptBoth_bit_less_MAE.pdf', bbox_inches='tight')
plt.show()